In [ ]:
from otoole import ReadCsv
import os

import xarray as xr
from numpy import inf

from otoole.utils import (
    _read_file,
    validate_config
)

import logging

logger = logging.getLogger(__name__)

config_path = "config.yaml"
folder_path = os.path.join('test', 'simplicity')
# folder_path = os.path.join('test', 'super_simple')

with open(config_path, "r") as config_file:
    config = _read_file(config_file, '.yaml')
validate_config(config)

read_strategy = ReadCsv(user_config=config)

model, defaults = read_strategy.read(folder_path)
logging.debug(model.keys())

In [ ]:
data_vars = {x: y.VALUE.to_xarray() for x, y in model.items() if config[x]['type'] == 'param'}
coords = {x: y.values.T[0] for x, y in model.items() if config[x]['type'] == 'set'}
ds = xr.Dataset(data_vars=data_vars, coords=coords)
ds = ds.assign_coords({'_REGION': model['REGION'].values.T[0]})

In [ ]:
for param, default in defaults.items():
    if default != 0:
        ds[param] = ds[param].fillna(default)

# Model Creation

In [ ]:
from linopy import Model, solvers, available_solvers
m = Model(force_dim_names=True)

## Variables

In [ ]:
RRTiFY = [ds.coords['REGION'], ds.coords['_REGION'], ds.coords['TIMESLICE'], ds.coords['FUEL'], ds.coords['YEAR']]
RRFY = [ds.coords['REGION'], ds.coords['_REGION'], ds.coords['FUEL'], ds.coords['YEAR']]
RLFY = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['FUEL'], ds.coords['YEAR']]
RFY = [ds.coords['REGION'], ds.coords['FUEL'], ds.coords['YEAR']]
RTiFY = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['FUEL'], ds.coords['YEAR']]
RSY = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['YEAR']]
RTeY = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['YEAR']]

mask = ~ds['SpecifiedAnnualDemand'].expand_dims('TIMESLICE').isnull()
rate_of_demand = m.add_variables(lower=0, upper=inf, coords=RTiFY, name='RateOfDemand', integer=False, mask=mask)
demand = m.add_variables(lower=0, upper=inf, coords=RTiFY, name='Demand', integer=False)

coords = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['SEASON'], ds.coords['DAYTYPE'], ds.coords['DAILYTIMEBRACKET'], ds.coords['YEAR']]
rate_of_storage_charge = m.add_variables(lower=-inf, upper=inf, coords=coords, name='RateOfStorageCharge', integer=False)
coords = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['SEASON'], ds.coords['DAYTYPE'], ds.coords['DAILYTIMEBRACKET'], ds.coords['YEAR']]
rate_of_storage_discharge = m.add_variables(lower=-inf, upper=inf, coords=coords, name='RateOfStorageDischarge', integer=False)
coords = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['SEASON'], ds.coords['DAYTYPE'], ds.coords['DAILYTIMEBRACKET'], ds.coords['YEAR']]
net_charge_within_year = m.add_variables(lower=-inf, upper=inf, coords=coords, name='NetChargeWithinYear', integer=False)
coords = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['SEASON'], ds.coords['DAYTYPE'], ds.coords['DAILYTIMEBRACKET'], ds.coords['YEAR']]
net_charge_within_day = m.add_variables(lower=-inf, upper=inf, coords=coords, name='NetChargeWithinDay', integer=False)

coords = RSY
storage_level_year_start = m.add_variables(lower=0, upper=inf, coords=coords, name='StorageLevelYearStart', integer=False)
coords = RSY
storage_level_year_finish = m.add_variables(lower=0, upper=inf, coords=coords, name='StorageLevelYearFinish', integer=False)
coords = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['SEASON'], ds.coords['YEAR']]
storage_level_season_start = m.add_variables(lower=0, upper=inf, coords=coords, name='StorageLevelSeasonStart', integer=False)
coords = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['SEASON'], ds.coords['DAYTYPE'], ds.coords['YEAR']]
storage_level_day_type_start = m.add_variables(lower=0, upper=inf, coords=coords, name='StorageLevelDayTypeStart', integer=False)
coords = [ds.coords['REGION'], ds.coords['STORAGE'], ds.coords['SEASON'], ds.coords['DAYTYPE'], ds.coords['YEAR']]
storage_level_day_type_finish = m.add_variables(lower=0, upper=inf, coords=coords, name='StorageLevelDayTypeFinish', integer=False)
coords = RSY
storage_lower_limit = m.add_variables(lower=0, upper=inf, coords=coords, name='StorageLowerLimit', integer=False)
coords = RSY
storage_upper_limit = m.add_variables(lower=0, upper=inf, coords=coords, name='StorageUpperLimit', integer=False)
coords = RSY
accumulated_new_storage_capacity = m.add_variables(lower=0, upper=inf, coords=coords, name='AccumulatedNewStorageCapacity', integer=False)
coords = RSY
new_storage_capacity = m.add_variables(lower=0, upper=inf, coords=coords, name='NewStorageCapacity', integer=False)
coords = RSY
capital_investment_storage = m.add_variables(lower=0, upper=inf, coords=coords, name='CapitalInvestmentStorage', integer=False)
coords = RSY
discounted_capital_investment_storage = m.add_variables(lower=0, upper=inf, coords=coords, name='DiscountedCapitalInvestmentStorage', integer=False)
coords = RSY
salvage_value_storage = m.add_variables(lower=0, upper=inf, coords=coords, name='SalvageValueStorage', integer=False)
coords = RSY
discounted_salvage_value_storage = m.add_variables(lower=0, upper=inf, coords=coords, name='DiscountedSalvageValueStorage', integer=False)
coords = RSY
total_discounted_storage_cost = m.add_variables(lower=0, upper=inf, coords=coords, name='TotalDiscountedStorageCost', integer=False)
coords = RTeY
number_of_new_technology_units = m.add_variables(lower=0, upper=inf, coords=coords, name='NumberOfNewTechnologyUnits', integer=True)
coords = RTeY
new_capacity = m.add_variables(lower=0, upper=inf, coords=coords, name='NewCapacity', integer=False)
coords = RTeY
accumulated_new_capacity = m.add_variables(lower=0, upper=inf, coords=coords, name='AccumulatedNewCapacity', integer=False)
coords = RTeY
total_capacity_annual = m.add_variables(lower=0, upper=inf, coords=coords, name='TotalCapacityAnnual', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['TECHNOLOGY'], ds.coords['MODE_OF_OPERATION'], ds.coords['YEAR']]
rate_of_activity = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfActivity', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['TIMESLICE'], ds.coords['YEAR']]
rate_of_total_activity = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfTotalActivity', integer=False)
coords = RTeY
total_technology_annual_activity = m.add_variables(lower=0, upper=inf, coords=coords, name='TotalTechnologyAnnualActivity', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['MODE_OF_OPERATION'], ds.coords['YEAR']]
total_annual_technology_activity_by_mode = m.add_variables(lower=0, upper=inf, coords=coords, name='TotalAnnualTechnologyActivityByMode', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY']]
total_technology_model_period_activity = m.add_variables(lower=-inf, upper=inf, coords=coords, name='TotalTechnologyModelPeriodActivity', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['TECHNOLOGY'], ds.coords['MODE_OF_OPERATION'], ds.coords['FUEL'], ds.coords['YEAR']]
rate_of_production_by_technology_by_mode = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfProductionByTechnologyByMode', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['TECHNOLOGY'], ds.coords['FUEL'], ds.coords['YEAR']]
rate_of_production_by_technology = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfProductionByTechnology', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['TECHNOLOGY'], ds.coords['FUEL'], ds.coords['YEAR']]
production_by_technology = m.add_variables(lower=0, upper=inf, coords=coords, name='ProductionByTechnology', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['FUEL'], ds.coords['YEAR']]
production_by_technology_annual = m.add_variables(lower=0, upper=inf, coords=coords, name='ProductionByTechnologyAnnual', integer=False)
coords = RTiFY
rate_of_production = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfProduction', integer=False)
coords = RTiFY
production = m.add_variables(lower=0, upper=inf, coords=coords, name='Production', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['TECHNOLOGY'], ds.coords['MODE_OF_OPERATION'], ds.coords['FUEL'], ds.coords['YEAR']]
rate_of_use_by_technology_by_mode = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfUseByTechnologyByMode', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['TECHNOLOGY'], ds.coords['FUEL'], ds.coords['YEAR']]
rate_of_use_by_technology = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfUseByTechnology', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['FUEL'], ds.coords['YEAR']]
use_by_technology_annual = m.add_variables(lower=0, upper=inf, coords=coords, name='UseByTechnologyAnnual', integer=False)
coords = RTiFY
rate_of_use = m.add_variables(lower=0, upper=inf, coords=coords, name='RateOfUse', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['TECHNOLOGY'], ds.coords['FUEL'], ds.coords['YEAR']]
use_by_technology = m.add_variables(lower=0, upper=inf, coords=coords, name='UseByTechnology', integer=False)
coords = RTiFY
use = m.add_variables(lower=0, upper=inf, coords=coords, name='Use', integer=False)
coords = [ds.coords['REGION'], ds.coords['_REGION'], ds.coords['TIMESLICE'], ds.coords['FUEL'], ds.coords['YEAR']]
trade = m.add_variables(lower=-inf, upper=inf, coords=coords, name='Trade', integer=False)
coords = [ds.coords['REGION'], ds.coords['_REGION'], ds.coords['FUEL'], ds.coords['YEAR']]
trade_annual = m.add_variables(lower=-inf, upper=inf, coords=coords, name='TradeAnnual', integer=False)
coords = [ds.coords['REGION'], ds.coords['FUEL'], ds.coords['YEAR']]
production_annual = m.add_variables(lower=0, upper=inf, coords=coords, name='ProductionAnnual', integer=False)
coords = [ds.coords['REGION'], ds.coords['FUEL'], ds.coords['YEAR']]
use_annual = m.add_variables(lower=0, upper=inf, coords=coords, name='UseAnnual', integer=False)
coords = RTeY
capital_investment = m.add_variables(lower=0, upper=inf, coords=coords, name='CapitalInvestment', integer=False)
coords = RTeY
discounted_capital_investment = m.add_variables(lower=0, upper=inf, coords=coords, name='DiscountedCapitalInvestment', integer=False)
coords = RTeY
salvage_value = m.add_variables(lower=0, upper=inf, coords=coords, name='SalvageValue', integer=False)
coords = RTeY
discounted_salvage_value = m.add_variables(lower=0, upper=inf, coords=coords, name='DiscountedSalvageValue', integer=False)
coords = RTeY
operating_cost = m.add_variables(lower=0, upper=inf, coords=coords, name='OperatingCost', integer=False)
coords = RTeY
discounted_operating_cost = m.add_variables(lower=0, upper=inf, coords=coords, name='DiscountedOperatingCost', integer=False)
coords = RTeY
annual_variable_operating_cost = m.add_variables(lower=0, upper=inf, coords=coords, name='AnnualVariableOperatingCost', integer=False)
coords = RTeY
annual_fixed_operating_cost = m.add_variables(lower=0, upper=inf, coords=coords, name='AnnualFixedOperatingCost', integer=False)
coords = RTeY
total_discounted_cost_by_technology = m.add_variables(lower=0, upper=inf, coords=coords, name='TotalDiscountedCostByTechnology', integer=False)
coords = [ds.coords['REGION'], ds.coords['YEAR']]
total_discounted_cost = m.add_variables(lower=0, upper=inf, coords=coords, name='TotalDiscountedCost', integer=False)
coords = [ds.coords['REGION']]
model_period_cost_by_region = m.add_variables(lower=0, upper=inf, coords=coords, name='ModelPeriodCostByRegion', integer=False)
coords = [ds.coords['REGION'], ds.coords['YEAR']]
total_capacity_in_reserve_margin = m.add_variables(lower=0, upper=inf, coords=coords, name='TotalCapacityInReserveMargin', integer=False)
coords = [ds.coords['REGION'], ds.coords['TIMESLICE'], ds.coords['YEAR']]
demand_needing_reserve_margin = m.add_variables(lower=0, upper=inf, coords=coords, name='DemandNeedingReserveMargin', integer=False)
coords = [ds.coords['REGION'], ds.coords['YEAR']]
total_r_e_production_annual = m.add_variables(lower=-inf, upper=inf, coords=coords, name='TotalREProductionAnnual', integer=False)
coords = [ds.coords['REGION'], ds.coords['YEAR']]
r_e_total_production_of_target_fuel_annual = m.add_variables(lower=-inf, upper=inf, coords=coords, name='RETotalProductionOfTargetFuelAnnual', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['EMISSION'], ds.coords['MODE_OF_OPERATION'], ds.coords['YEAR']]
annual_technology_emission_by_mode = m.add_variables(lower=0, upper=inf, coords=coords, name='AnnualTechnologyEmissionByMode', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['EMISSION'], ds.coords['YEAR']]
annual_technology_emission = m.add_variables(lower=0, upper=inf, coords=coords, name='AnnualTechnologyEmission', integer=False)
coords = [ds.coords['REGION'], ds.coords['TECHNOLOGY'], ds.coords['EMISSION'], ds.coords['YEAR']]
annual_technology_emission_penalty_by_emission = m.add_variables(lower=0, upper=inf, coords=coords, name='AnnualTechnologyEmissionPenaltyByEmission', integer=False)
coords = RTeY
annual_technology_emissions_penalty = m.add_variables(lower=0, upper=inf, coords=coords, name='AnnualTechnologyEmissionsPenalty', integer=False)
coords = RTeY
discounted_technology_emissions_penalty = m.add_variables(lower=0, upper=inf, coords=coords, name='DiscountedTechnologyEmissionsPenalty', integer=False)
coords = [ds.coords['REGION'], ds.coords['EMISSION'], ds.coords['YEAR']]
annual_emissions = m.add_variables(lower=0, upper=inf, coords=coords, name='AnnualEmissions', integer=False)
coords = [ds.coords['REGION'], ds.coords['EMISSION']]
model_period_emissions = m.add_variables(lower=0, upper=inf, coords=coords, name='ModelPeriodEmissions', integer=False)

# Discounting

```ampl
param DiscountRate{r in REGION};
param DiscountRateIdv{r in REGION, t in TECHNOLOGY}, default DiscountRate[r];

param DiscountFactor{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMid{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.5);

param OperationalLife{r in REGION, t in TECHNOLOGY};

param CapitalRecoveryFactor{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRateIdv[r,t])^(-1))/(1 - (1 + DiscountRateIdv[r,t])^(-(OperationalLife[r,t])));
param PvAnnuity{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRate[r])^(-(OperationalLife[r,t]))) * (1 + DiscountRate[r]) / DiscountRate[r];

param DiscountRateStorage{r in REGION, s in STORAGE};
param DiscountFactorStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMidStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.5);
```

In [ ]:
discount_factor = ((1 + ds['DiscountRate']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid = ((1 + ds['DiscountRate']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

In [ ]:
discount_factor_idv = ((1 + ds['DiscountRateIdv']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid_idv = ((1 + ds['DiscountRateIdv']) ** -(ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

In [ ]:
pv_annuity = (1 - (1 + ds['DiscountRate'])**(-(ds['OperationalLife']))) * (1 + ds['DiscountRate']) / ds['DiscountRate']

In [ ]:
capital_recovery_factor = (1 - (1 + ds['DiscountRateIdv'])**(-1))/(1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife'])))
capital_recovery_factor

# Constraints

## Demand

```ampl
s.t. EQ_SpecifiedDemand{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
						SpecifiedAnnualDemand[r,f,y] <> 0}:
	SpecifiedAnnualDemand[r,f,y] * SpecifiedDemandProfile[r,f,l,y] / YearSplit[l,y]
	=
	RateOfDemand[r,l,f,y];
	```

In [ ]:
mask = ~ds['SpecifiedAnnualDemand'].isnull()
con = rate_of_demand == (ds['SpecifiedAnnualDemand'] * ds['SpecifiedDemandProfile'] / ds['YearSplit'])
eq_specified_demand = m.add_constraints(con, name='EQ_SpecifiedDemand', mask=mask)

In [ ]:
eq_specified_demand

## Capacity Adequacy A

```ampl
s.t. CAa1_TotalNewCapacity{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AccumulatedNewCapacity[r,t,y]
	=
	sum{yy in YEAR: y-yy < OperationalLife[r,t] && y - yy >= 0} NewCapacity[r,t,yy];
```

In [ ]:
def bounds(model, r, t, y):
    return accumulated_new_capacity[r,t,y] - 1 * \
                sum(new_capacity[r,t,yy] for yy in ds.coords['YEAR'].values 
                    if (y-yy >= 0) and (y-yy < ds['OperationalLife'].sel({'REGION':r, 'TECHNOLOGY': t}))) == 0
    
c_aa1_total_new_capacity = m.add_constraints(bounds, coords=RTeY, name='CAa1_TotalNewCapacity')

In [ ]:
c_aa1_total_new_capacity

```ampl
s.t. CAa2_TotalAnnualCapacity{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AccumulatedNewCapacity[r,t,y] + ResidualCapacity[r,t,y]
	=
	TotalCapacityAnnual[r,t,y];
```

In [ ]:
con = accumulated_new_capacity - total_capacity_annual
caa2_total_annual_capacity = m.add_constraints(lhs=con, sign='==', rhs=-ds['ResidualCapacity'].fillna(0), name="CAa2_TotalAnnualCapacity", coords=RTeY)

In [ ]:
caa2_total_annual_capacity

```ampl
s.t. CAa3_TotalActivityOfEachTechnology{r in REGION, t in TECHNOLOGY, l in TIMESLICE, y in YEAR}:
	sum{m in MODE_OF_OPERATION} RateOfActivity[r,l,t,m,y]
	=
	RateOfTotalActivity[r,t,l,y];
```

In [ ]:
con = rate_of_activity.sum(dims='MODE_OF_OPERATION') - rate_of_total_activity == 0
caa3_total_activity_of_each_technology = m.add_constraints(con, name='CAa3_TotalActivityOfEachTechnology')

In [ ]:
caa3_total_activity_of_each_technology

```ampl
s.t. CAa4_Constraint_Capacity{r in REGION, l in TIMESLICE, t in TECHNOLOGY, y in YEAR}:
	RateOfTotalActivity[r,t,l,y]
	<=
	TotalCapacityAnnual[r,t,y] * CapacityFactor[r,t,l,y] * CapacityToActivityUnit[r,t];
```

In [ ]:
con = rate_of_total_activity - (total_capacity_annual * ds['CapacityFactor'] * ds['CapacityToActivityUnit']) <=0
mask = ~ds['CapacityFactor'].isnull()
caa4_constraint_capacity = m.add_constraints(con, name='CAa4_Constraint_Capacity', mask=mask)

In [ ]:
caa4_constraint_capacity

```ampl
s.t. CAa5_TotalNewCapacity{r in REGION, t in TECHNOLOGY, y in YEAR: CapacityOfOneTechnologyUnit[r,t,y]<>0}:
	CapacityOfOneTechnologyUnit[r,t,y] * NumberOfNewTechnologyUnits[r,t,y]
	=
	NewCapacity[r,t,y];
```

In [ ]:
con = ds['CapacityOfOneTechnologyUnit'] * number_of_new_technology_units - new_capacity == 0
mask = ~ds['CapacityOfOneTechnologyUnit'].isnull()
caa5_total_new_capacity = m.add_constraints(con, name='CAa5_TotalNewCapacity', mask=mask)

In [ ]:
caa5_total_new_capacity

## Capacity Adequacy B
```ampl
s.t. CAb1_PlannedMaintenance{r in REGION, t in TECHNOLOGY, y in YEAR: AvailabilityFactor[r,t,y] < 1}:
	sum{l in TIMESLICE} RateOfTotalActivity[r,t,l,y] * YearSplit[l,y]
	<=
	sum{l in TIMESLICE} (TotalCapacityAnnual[r,t,y] * CapacityFactor[r,t,l,y] * YearSplit[l,y])
	* AvailabilityFactor[r,t,y] * CapacityToActivityUnit[r,t];
```

In [ ]:
mask = ds['AvailabilityFactor'] < 1
con = (rate_of_total_activity * ds['YearSplit']).sum(dims='TIMESLICE') \
      - (total_capacity_annual * ds['CapacityFactor'] * ds['YearSplit']).sum(dims='TIMESLICE') * ds['AvailabilityFactor'] * ds['CapacityToActivityUnit'] == 0
cab1_planned_maintenance = m.add_constraints(con, name='CAb1_PlannedMaintenance', coords=RTeY, mask=mask)

In [ ]:
cab1_planned_maintenance

## Energy Balance A

```ampl
s.t. EBa1_RateOfFuelProduction1{
	r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, m in MODE_OF_OPERATION, y in YEAR:
	OutputActivityRatio[r,t,f,m,y] <> 0}:
	RateOfActivity[r,l,t,m,y] * OutputActivityRatio[r,t,f,m,y]
	=
	RateOfProductionByTechnologyByMode[r,l,t,m,f,y];
```

In [ ]:
mask = ~ds['OutputActivityRatio'].isnull()
con = rate_of_activity * ds['OutputActivityRatio'] - rate_of_production_by_technology_by_mode == 0
eba1_rate_of_fuel_production1 = m.add_constraints(con, name='EBa1_RateOfFuelProduction1', mask=mask)

In [ ]:
eba1_rate_of_fuel_production1

```ampl
s.t. EBa2_RateOfFuelProduction2{r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, y in YEAR}:
	sum{m in MODE_OF_OPERATION: OutputActivityRatio[r,t,f,m,y] <> 0} RateOfProductionByTechnologyByMode[r,l,t,m,f,y]
	=
	RateOfProductionByTechnology[r,l,t,f,y];
```

In [ ]:
mask = ds['OutputActivityRatio'].sum(dim='MODE_OF_OPERATION') != 0
con = (rate_of_production_by_technology_by_mode).sum(dims='MODE_OF_OPERATION') - rate_of_production_by_technology == 0
eba2_rate_of_fuel_production2 = m.add_constraints(con, name='EBa2_RateOfFuelProduction2', mask=mask)

In [ ]:
eba2_rate_of_fuel_production2

```ampl
s.t. EBa3_RateOfFuelProduction3{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							    (sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} OutputActivityRatio[r,t,f,m,y]) <> 0}:
	sum{t in TECHNOLOGY} RateOfProductionByTechnology[r,l,t,f,y]
	=
	RateOfProduction[r,l,f,y];
```

In [ ]:
con = rate_of_production_by_technology.sum(dims='TECHNOLOGY') - rate_of_production == 0
mask = ~ds['OutputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']).isnull()
eba3_rate_of_fuel_production3 = m.add_constraints(con, name='EBa3_RateOfFuelProduction3', mask=mask)

In [ ]:
eba3_rate_of_fuel_production3

```ampl
s.t. EBa4_RateOfFuelUse1{r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, m in MODE_OF_OPERATION, y in YEAR:
						 InputActivityRatio[r,t,f,m,y] <> 0}:
	RateOfActivity[r,l,t,m,y] * InputActivityRatio[r,t,f,m,y]
	=
	RateOfUseByTechnologyByMode[r,l,t,m,f,y];
```

In [ ]:
con = rate_of_activity * ds['InputActivityRatio'] - rate_of_production_by_technology_by_mode == 0
mask = ~ds['InputActivityRatio'].isnull()
eba4_rate_of_fuel_use1 = m.add_constraints(con, name='EBa4_RateOfFuelUse1', mask=mask)

In [ ]:
eba4_rate_of_fuel_use1

```ampl
s.t. EBa5_RateOfFuelUse2{r in REGION, l in TIMESLICE, f in FUEL, t in TECHNOLOGY, y in YEAR:
						 sum{m in MODE_OF_OPERATION} InputActivityRatio[r,t,f,m,y] <> 0}:
	sum{m in MODE_OF_OPERATION: InputActivityRatio[r,t,f,m,y] <> 0}
	RateOfUseByTechnologyByMode[r,l,t,m,f,y]
	=
	RateOfUseByTechnology[r,l,t,f,y];
```

In [ ]:
con = rate_of_use_by_technology_by_mode.sum(dims='MODE_OF_OPERATION') - rate_of_use_by_technology == 0
mask = ds['InputActivityRatio'].sum(dim='MODE_OF_OPERATION') != 0
eba5_rate_of_fuel_use2 = m.add_constraints(con, name='EBa5_RateOfFuelUse2', mask=mask)

In [ ]:
eba5_rate_of_fuel_use2

```ampl
s.t. EBa6_RateOfFuelUse3{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
						 sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} InputActivityRatio[r,t,f,m,y] <> 0}:
	sum{t in TECHNOLOGY} RateOfUseByTechnology[r,l,t,f,y]
	=
	RateOfUse[r,l,f,y];
```

In [ ]:
con = rate_of_use_by_technology.sum(dims='TECHNOLOGY') - rate_of_use == 0
mask = ds['InputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']) != 0
eba6_rate_of_fuel_use3 = m.add_constraints(con, name='EBa6_RateOfFuelUse3', mask=mask)

In [ ]:
eba6_rate_of_fuel_use3

```ampl
s.t. EBa7_EnergyBalanceEachTS1{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							   (sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} OutputActivityRatio[r,t,f,m,y]) <> 0}:
	RateOfProduction[r,l,f,y] * YearSplit[l,y]
	=
	Production[r,l,f,y];
```

In [ ]:
con = rate_of_production * ds['YearSplit'] - production == 0
mask = ds['OutputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']) != 0
eba7_energy_balance_each_ts1 = m.add_constraints(con, name='EBa7_EnergyBalanceEachTS1', mask=mask)

In [ ]:
eba7_energy_balance_each_ts1

```ampl
s.t. EBa8_EnergyBalanceEachTS2{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							   (sum{t in TECHNOLOGY, m in MODE_OF_OPERATION} InputActivityRatio[r,t,f,m,y]) <> 0}:
	RateOfUse[r,l,f,y] * YearSplit[l,y]
	=
	Use[r,l,f,y];
```

In [ ]:
con = rate_of_use * ds['YearSplit'] - use == 0
mask = ds['InputActivityRatio'].sum(dim=['TECHNOLOGY', 'MODE_OF_OPERATION']) != 0
eba8_energy_balance_each_ts2 = m.add_constraints(con, name='EBa8_EnergyBalanceEachTS2', mask=mask)

In [ ]:
eba8_energy_balance_each_ts2

```ampl
s.t. EBa9_EnergyBalanceEachTS3{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
							   SpecifiedAnnualDemand[r,f,y] <> 0}:
	RateOfDemand[r,l,f,y] * YearSplit[l,y]
	=
	Demand[r,l,f,y];
```

In [ ]:
con = rate_of_demand * ds['YearSplit'] - demand == 0
mask = ~ds['SpecifiedAnnualDemand'].isnull()
eba9_energy_balance_each_ts3 = m.add_constraints(con, name='EBa9_EnergyBalanceEachTS3', mask=mask)

In [ ]:
eba9_energy_balance_each_ts3

```ampl
s.t. EBa10_EnergyBalanceEachTS4{r in REGION, rr in REGION, l in TIMESLICE, f in FUEL, y in YEAR:
								TradeRoute[r,rr,f,y] <> 0}:
	Trade[r,rr,l,f,y]
	=
	-Trade[rr,r,l,f,y];
```

```python
def EnergyBalanceEachTS4_rule(model, r, rr, l, f, y):
    return model.Trade[r, rr, l, f, y] + model.Trade[rr, r, l, f, y] == 0
```

In [ ]:
def energy_balance_each_ts4_rule(model, r, rr, l, f, y):
    return trade[r,rr,l,f,y] + trade[rr,r,l,f,y] == 0
tr = ds['TradeRoute']
mask = ~tr.where(tr.REGION != tr._REGION).isnull()
eba10_energy_balance_each_ts4 = m.add_constraints(energy_balance_each_ts4_rule, name='EBa10_EnergyBalanceEachTS4', mask=mask, coords=RRTiFY)

In [ ]:
eba10_energy_balance_each_ts4

```ampl
s.t. EBa11_EnergyBalanceEachTS5{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR}:
	Production[r,l,f,y]
	>=
	Demand[r,l,f,y] + Use[r,l,f,y] + sum{rr in REGION} Trade[r,rr,l,f,y] * TradeRoute[r,rr,f,y];
```

In [ ]:
def bounds(model, r, l, f, y):
    # if ds['TradeRoute'].sel({'REGION': r,
    #                          'TIMESLICE': l,
    #                          'FUEL': f,
    #                          'YEAR': y}).sum('_REGION') != 0:
    #     return production[r,l,f,y] - (demand[r,l,f,y] + use[r,l,f,y] + sum(trade[r,rr,l,f,y] * ds['TradeRoute'][r,rr,l,f,y] for rr in ds.coords['_REGION'])) >= 0
    # else:
    return production[r,l,f,y] - (demand[r,l,f,y] + use[r,l,f,y]) >= 0

eba11_energy_balance_each_ts5 = m.add_constraints(bounds, name='EBa11_EnergyBalanceEachTS5', coords=RLFY)

In [ ]:
eba11_energy_balance_each_ts5

# Energy Balance B

```ampl
s.t. EBb1_EnergyBalanceEachYear1{r in REGION, f in FUEL, y in YEAR}:
	sum{l in TIMESLICE} Production[r,l,f,y]
	=
	ProductionAnnual[r,f,y];
```

In [ ]:
con = production.sum(dims='TIMESLICE') - production_annual == 0
ebb1_energy_balance_each_year1 = m.add_constraints(con, name='EBb1_EnergyBalanceEachYear1')

In [ ]:
ebb1_energy_balance_each_year1

```ampl
s.t. EBb2_EnergyBalanceEachYear2{r in REGION, f in FUEL, y in YEAR}:
	sum{l in TIMESLICE} Use[r,l,f,y]
	=
	UseAnnual[r,f,y];
```

In [ ]:
con = use.sum('TIMESLICE') - use_annual == 0
ebb2_energy_balance_each_year2 = m.add_constraints(con, name='EBb2_EnergyBalanceEachYear2')

In [ ]:
ebb2_energy_balance_each_year2

```ampl
s.t. EBb3_EnergyBalanceEachYear3{r in REGION, rr in REGION, f in FUEL, y in YEAR}:
	sum{l in TIMESLICE} Trade[r,rr,l,f,y]
	=
	TradeAnnual[r,rr,f,y];
```

In [ ]:
# def ebb3_energy_balance_each_year3_rule(model, r, rr, f, y):
#     if rr != r:
#         return trade.sum('TIMESLICE')[r, rr, f, y] - trade_annual[r, rr, f, y] == 0
# # con = trade.sum('TIMESLICE') - trade_annual == 0
# tr = trade
# mask = ~tr.where(tr.REGION != tr._REGION).isnull()
# ebb3_energy_balance_each_year3 = m.add_constraints(ebb3_energy_balance_each_year3_rule, name='EBb3_EnergyBalanceEachYear3', coords=RRFY, mask=mask)

In [ ]:
# ebb3_energy_balance_each_year3

```ampl
s.t. EBb4_EnergyBalanceEachYear4{r in REGION, f in FUEL, y in YEAR}:
	ProductionAnnual[r,f,y]
	>=
	UseAnnual[r,f,y] + sum{rr in REGION} TradeAnnual[r,rr,f,y] * TradeRoute[r,rr,f,y] + AccumulatedAnnualDemand[r,f,y];
```

In [ ]:
con = production_annual - use_annual - (trade_annual.sum('_REGION') * ds['TradeRoute'].sum('_REGION'))
# mask = ~ds['AccumulatedAnnualDemand'].isnull()
ebb4_energy_balance_each_year4 = m.add_constraints(lhs=con, sign='>=', rhs=ds['AccumulatedAnnualDemand'].fillna(0), name='EBb4_EnergyBalanceEachYear4')

In [ ]:
ebb4_energy_balance_each_year4

# Accounting Technology Production/Use

```ampl
s.t. Acc1_FuelProductionByTechnology{r in REGION, l in TIMESLICE, t in TECHNOLOGY, f in FUEL, y in YEAR}:
	RateOfProductionByTechnology[r,l,t,f,y] * YearSplit[l,y]
	=
	ProductionByTechnology[r,l,t,f,y];
```

In [ ]:
con = (rate_of_production_by_technology * ds['YearSplit']) - production_by_technology == 0
mask = ds['OutputActivityRatio'].sum('MODE_OF_OPERATION') != 0
acc1_fuel_production_by_technology = m.add_constraints(con, name='Acc1_FuelProductionByTechnology', mask=mask)

In [ ]:
acc1_fuel_production_by_technology

```ampl
s.t. Acc2_FuelUseByTechnology{r in REGION, l in TIMESLICE, t in TECHNOLOGY, f in FUEL, y in YEAR}:
	RateOfUseByTechnology[r,l,t,f,y] * YearSplit[l,y]
	=
	UseByTechnology[r,l,t,f,y];
```

In [ ]:
con = rate_of_use_by_technology * ds['YearSplit'] - use_by_technology == 0
mask = ds['InputActivityRatio'].sum('MODE_OF_OPERATION') != 0
acc2_fuel_use_by_technology = m.add_constraints(con, name='Acc2_FuelUseByTechnology', mask=mask)

In [ ]:
acc2_fuel_use_by_technology

```ampl
s.t. Acc3_AverageAnnualRateOfActivity{r in REGION, t in TECHNOLOGY, m in MODE_OF_OPERATION, y in YEAR}:
	sum{l in TIMESLICE} RateOfActivity[r,l,t,m,y]*YearSplit[l,y]
	=
	TotalAnnualTechnologyActivityByMode[r,t,m,y];
```

In [ ]:
con = rate_of_activity.sum('TIMESLICE') * ds['YearSplit'] - total_annual_technology_activity_by_mode == 0
mask = ds['OutputActivityRatio'].sum('FUEL') != 0
acc3_average_annual_rate_of_activity = m.add_constraints(con, name='Acc3_AverageAnnualRateOfActivity', mask=mask)

In [ ]:
acc3_average_annual_rate_of_activity

```ampl
s.t. Acc4_ModelPeriodCostByRegion{r in REGION}:
	sum{y in YEAR}TotalDiscountedCost[r,y] = ModelPeriodCostByRegion[r];
```

In [ ]:
con = total_discounted_cost.sum('YEAR') - model_period_cost_by_region == 0
acc4_model_period_cost_by_region = m.add_constraints(con, name='Acc4_ModelPeriodCostByRegion')

In [ ]:
acc4_model_period_cost_by_region

## Capital Costs

```ampl
s.t. CC1_UndiscountedCapitalInvestment{r in REGION, t in TECHNOLOGY, y in YEAR}: 
        CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] 
        = 
        CapitalInvestment[r,t,y];
```

In [ ]:
con = ds['CapitalCost'] * new_capacity * capital_recovery_factor * pv_annuity - capital_investment == 0
mask = ~ds['CapitalCost'].isnull()
cc1_undiscounted_capital_investment = m.add_constraints(con, name='CC1_UndiscountedCapitalInvestment', mask=mask)

In [ ]:
cc1_undiscounted_capital_investment

```ampl
s.t. CC2_DiscountingCapitalInvestment{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    CapitalInvestment[r,t,y]  / DiscountFactor[r,y] = DiscountedCapitalInvestment[r,t,y];
```

In [ ]:
con = capital_investment / ds['DiscountRate'] - discounted_capital_investment == 0
mask = ~ds['CapitalCost'].isnull()
cc2_discounting_capital_investment = m.add_constraints(con, name='CC2_DiscountingCapitalInvestment', mask=mask)

In [ ]:
cc2_discounting_capital_investment

## Salvage Value

### GNU MathProg implementation
```ampl
s.t. SV1_SalvageValueAtEndOfPeriod1{
    r in REGION, t in TECHNOLOGY, y in YEAR: 
        DepreciationMethod[r]=1 && 
        (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)) && 
        DiscountRate[r]>0}: 
    SalvageValue[r,t,y] 
    = 
    CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] * 
    (1-(((1+DiscountRate[r])^(max{yy in YEAR} max(yy) - y+1)-1)/((1+DiscountRate[r])^OperationalLife[r,t]-1)));
```
### Pyomo implementation
```python
def SalvageValueAtEndOfPeriod1_rule(model, r, t, y):
    if (
        model.DepreciationMethod[r] == 1
        and ((y + model.OperationalLife[r, t] - 1) > max(model.YEAR))
        and model.DiscountRate[r] > 0
    ):
        return model.SalvageValue[r, t, y] == model.CapitalCost[
            r, t, y
        ] * model.NewCapacity[r, t, y] * (
            1
            - (
                ((1 + model.DiscountRate[r]) ** (max(model.YEAR) - y + 1) - 1)
                / ((1 + model.DiscountRate[r]) ** model.OperationalLife[r, t] - 1)
            )
        )
    elif (
        model.DepreciationMethod[r] == 1
        and ((y + model.OperationalLife[r, t] - 1) > max(model.YEAR))
        and model.DiscountRate[r] == 0
    ) or (
        model.DepreciationMethod[r] == 2
        and (y + model.OperationalLife[r, t] - 1) > (max(model.YEAR))
    ):
        return model.SalvageValue[r, t, y] == model.CapitalCost[
            r, t, y
        ] * model.NewCapacity[r, t, y] * (
            1 - (max(model.YEAR) - y + 1) / model.OperationalLife[r, t]
        )
    else:
        return model.SalvageValue[r, t, y] == 0


model.SalvageValueAtEndOfPeriod1 = Constraint(
    model.REGION, model.TECHNOLOGY, model.YEAR, rule=SalvageValueAtEndOfPeriod1_rule
)
```

In [ ]:
# con = salvage_value = ds['CapitalCost'] * new_capacity * capital_recovery_factor * pv_annuity 
# mask = ~ds['CapitalCost'].isnull()
# sv1_salvage_value_at_end_of_period1 = m.add_constraints(con, name='SV1_SalvageValueAtEndOfPeriod1', mask=mask)

In [ ]:
# sv1_salvage_value_at_end_of_period1

```ampl
s.t. SV2_SalvageValueAtEndOfPeriod2{r in REGION, t in TECHNOLOGY, y in YEAR: (DepreciationMethod[r]=1 && (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)) && DiscountRate[r]=0) || (DepreciationMethod[r]=2 && (y + OperationalLife[r,t]-1) > (max{yy in YEAR} max(yy)))}: SalvageValue[r,t,y] = CapitalCost[r,t,y] * NewCapacity[r,t,y] * CapitalRecoveryFactor[r,t] * PvAnnuity[r,t] *(1-(max{yy in YEAR} max(yy) - y+1)/OperationalLife[r,t]);
```

```ampl
s.t. SV3_SalvageValueAtEndOfPeriod3{r in REGION, t in TECHNOLOGY, y in YEAR: (y + OperationalLife[r,t]-1) <= (max{yy in YEAR} max(yy))}: 
    SalvageValue[r,t,y] = 0;
```

In [ ]:
con = salvage_value == 0
sv3_salvage_value_at_end_of_period3 = m.add_constraints(con, name='SV3_SalvageValueAtEndOfPeriod3')

In [ ]:
sv3_salvage_value_at_end_of_period3

```ampl
s.t. SV4_SalvageValueDiscountedToStartYear{r in REGION, t in TECHNOLOGY, y in YEAR}: DiscountedSalvageValue[r,t,y] = SalvageValue[r,t,y]/((1+DiscountRate[r])^(1+max{yy in YEAR} max(yy)-min{yy in YEAR} min(yy)));
```

In [ ]:
con = discounted_salvage_value - salvage_value == 0
sv4_salvage_value_discounted_to_start_year = m.add_constraints(con, name='SV4_SalvageValueDiscountedToStartYear')

In [ ]:
sv4_salvage_value_discounted_to_start_year

## Operating Costs

```ampl
s.t. OC1_OperatingCostsVariable{r in REGION, t in TECHNOLOGY, l in TIMESLICE, y in YEAR: sum{m in MODE_OF_OPERATION} VariableCost[r,t,m,y] <> 0}:
	sum{m in MODE_OF_OPERATION}
	TotalAnnualTechnologyActivityByMode[r,t,m,y] * VariableCost[r,t,m,y]
	=
	AnnualVariableOperatingCost[r,t,y];
```

In [ ]:
con = total_annual_technology_activity_by_mode.sum(dims='MODE_OF_OPERATION') * ds['VariableCost'] - annual_variable_operating_cost == 0
mask = (ds['VariableCost'].sum(dim='MODE_OF_OPERATION') != 0) & (~ds['VariableCost'].isnull())
oc1_operating_costs_variable = m.add_constraints(con, name='OC1_OperatingCostsVariable', mask=mask)

In [ ]:
oc1_operating_costs_variable

```ampl
s.t. OC2_OperatingCostsFixedAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	TotalCapacityAnnual[r,t,y]*FixedCost[r,t,y]
	=
	AnnualFixedOperatingCost[r,t,y];
```

In [ ]:
con = total_capacity_annual * ds['FixedCost'] - annual_fixed_operating_cost == 0
mask = ~ds['FixedCost'].isnull()
oc2_operating_costs_fixed_annual = m.add_constraints(con, name='OC2_OperatingCostsFixedAnnual', mask=mask)

In [ ]:
oc2_operating_costs_fixed_annual

```ampl
s.t. OC3_OperatingCostsTotalAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	AnnualFixedOperatingCost[r,t,y] + AnnualVariableOperatingCost[r,t,y]
	=
	OperatingCost[r,t,y];
```

In [ ]:
con = annual_fixed_operating_cost + annual_variable_operating_cost - operating_cost == 0
mask = (ds['VariableCost'].sum(dim='MODE_OF_OPERATION') != 0) & (~ds['FixedCost'].isnull())
oc3_operating_costs_total_annual = m.add_constraints(con, name='OC3_OperatingCostsTotalAnnual', mask=mask)

In [ ]:
oc3_operating_costs_total_annual

```ampl
s.t. OC4_DiscountedOperatingCostsTotalAnnual{r in REGION, t in TECHNOLOGY, y in YEAR}:
	OperatingCost[r,t,y] / DiscountFactorMid[r, y]
	=
	DiscountedOperatingCost[r,t,y];
```

In [ ]:
discount_factor_mid = 1
con = operating_cost / discount_factor_mid - discounted_operating_cost == 0
oc4_discounted_operating_costs_total_annual = m.add_constraints(con, name='OC4_DiscountedOperatingCostsTotalAnnual')

In [ ]:
oc4_discounted_operating_costs_total_annual

## Total Discounted Costs

```ampl
s.t. TDC1_TotalDiscountedCostByTechnology{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    DiscountedOperatingCost[r,t,y] + DiscountedCapitalInvestment[r,t,y] 
    + DiscountedTechnologyEmissionsPenalty[r,t,y] - DiscountedSalvageValue[r,t,y] = TotalDiscountedCostByTechnology[r,t,y];
```

In [ ]:
con = discounted_operating_cost + discounted_capital_investment + discounted_technology_emissions_penalty \
        - discounted_salvage_value - total_discounted_cost_by_technology == 0
tdc1_total_discounted_cost_by_technology = m.add_constraints(con, name='TDC1_TotalDiscountedCostByTechnology')

In [ ]:
tdc1_total_discounted_cost_by_technology

```ampl
s.t. TDC2_TotalDiscountedCost{r in REGION, y in YEAR}: 
    sum{t in TECHNOLOGY} TotalDiscountedCostByTechnology[r,t,y] + 
    sum{s in STORAGE} TotalDiscountedStorageCost[r,s,y] 
    = TotalDiscountedCost[r,y];
```

In [ ]:
con = total_discounted_cost_by_technology.sum('TECHNOLOGY') + total_discounted_storage_cost.sum('STORAGE') - total_discounted_cost == 0
tdc2_total_discounted_cost = m.add_constraints(con, name='TDC2_TotalDiscountedCost')

In [ ]:
tdc2_total_discounted_cost

## Total Capacity Constraints

```ampl
s.t. TCC1_TotalAnnualMaxCapacityConstraint{r in REGION, t in TECHNOLOGY, y in YEAR: TotalAnnualMaxCapacity[r,t,y] <> -1}: TotalCapacityAnnual[r,t,y] <= TotalAnnualMaxCapacity[r,t,y];
```

In [ ]:
con = total_capacity_annual <= ds['TotalAnnualMaxCapacity']
mask = ds['TotalAnnualMaxCapacity'] >= 0
tcc1_total_annual_max_capacity_constraint = m.add_constraints(con, name='TCC1_TotalAnnualMaxCapacityConstraint', mask=mask)

In [ ]:
tcc1_total_annual_max_capacity_constraint

```ampl
s.t. TCC2_TotalAnnualMinCapacityConstraint{r in REGION, t in TECHNOLOGY, y in YEAR: TotalAnnualMinCapacity[r,t,y]>0}: TotalCapacityAnnual[r,t,y] >= TotalAnnualMinCapacity[r,t,y];
```

In [ ]:
con = total_capacity_annual >= ds['TotalAnnualMinCapacity']
mask = ds['TotalAnnualMinCapacity'] > 0
tcc2_total_annual_min_capacity_constraint = m.add_constraints(con, name='TCC2_TotalAnnualMinCapacityConstraint', mask=mask)

In [ ]:
tcc2_total_annual_min_capacity_constraint

# Annual Activity Constraints

```ampl
s.t. AAC1_TotalAnnualTechnologyActivity{r in REGION, t in TECHNOLOGY, y in YEAR}: 
    sum{l in TIMESLICE} RateOfTotalActivity[r,t,l,y]*YearSplit[l,y] = TotalTechnologyAnnualActivity[r,t,y];
```



In [ ]:
con = (rate_of_total_activity * ds['YearSplit']).sum('TIMESLICE') - total_technology_annual_activity == 0
aac1_total_annual_technology_activity = m.add_constraints(con, name='AAC1_TotalAnnualTechnologyActivity')

In [ ]:
aac1_total_annual_technology_activity

```ampl
s.t. AAC2_TotalAnnualTechnologyActivityUpperLimit{r in REGION, t in TECHNOLOGY, y in YEAR: TotalTechnologyAnnualActivityUpperLimit[r,t,y] <> -1}:
    TotalTechnologyAnnualActivity[r,t,y] <= TotalTechnologyAnnualActivityUpperLimit[r,t,y] ;
```

In [ ]:
con = total_technology_annual_activity <= ds['TotalTechnologyAnnualActivityUpperLimit']
mask = ds['TotalTechnologyAnnualActivityUpperLimit'] != -1
aac2_total_annual_technology_activity_upper_limit = m.add_constraints(con, name='AAC2_TotalAnnualTechnologyActivityUpperLimit', mask=mask)

In [ ]:
aac2_total_annual_technology_activity_upper_limit

```ampl
s.t. AAC3_TotalAnnualTechnologyActivityLowerLimit{r in REGION, t in TECHNOLOGY, y in YEAR: TotalTechnologyAnnualActivityLowerLimit[r,t,y]>0}: TotalTechnologyAnnualActivity[r,t,y] >= TotalTechnologyAnnualActivityLowerLimit[r,t,y] ;
```

In [ ]:
con = total_technology_annual_activity >= ds['TotalTechnologyAnnualActivityLowerLimit']
mask = ds['TotalTechnologyAnnualActivityLowerLimit'] > 0
aac3_total_annual_technology_activity_lower_limit = m.add_constraints(con, name='AAC3_TotalAnnualTechnologyActivityLowerLimit', mask=mask)

In [ ]:
aac3_total_annual_technology_activity_lower_limit

# Total Activity Constraints

```ampl
s.t. TAC1_TotalModelHorizonTechnologyActivity{r in REGION, t in TECHNOLOGY}: 
    sum{y in YEAR} TotalTechnologyAnnualActivity[r,t,y] = TotalTechnologyModelPeriodActivity[r,t];
```

In [ ]:
con = total_technology_annual_activity.sum('YEAR') - total_technology_model_period_activity == 0
tac1_total_model_horizon_technology_activity = m.add_constraints(con, name='TAC1_TotalModelHorizonTechnologyActivity')

In [ ]:
tac1_total_model_horizon_technology_activity

```ampl
s.t. TAC2_TotalModelHorizonTechnologyActivityUpperLimit{r in REGION, t in TECHNOLOGY: TotalTechnologyModelPeriodActivityUpperLimit[r,t]<>-1}: TotalTechnologyModelPeriodActivity[r,t] <= TotalTechnologyModelPeriodActivityUpperLimit[r,t] ;
```

In [ ]:
con = total_technology_model_period_activity <= ds['TotalTechnologyModelPeriodActivityUpperLimit']
mask = ds['TotalTechnologyModelPeriodActivityUpperLimit'] != -1
tac2_total_model_horizon_technology_activity_upper_limit = m.add_constraints(con, name='TAC2_TotalModelHorizonTechnologyActivityUpperLimit', mask=mask)

In [ ]:
tac2_total_model_horizon_technology_activity_upper_limit

```ampl
s.t. TAC3_TotalModelHorizenTechnologyActivityLowerLimit{r in REGION, t in TECHNOLOGY: TotalTechnologyModelPeriodActivityLowerLimit[r,t]>0}: TotalTechnologyModelPeriodActivity[r,t] >= TotalTechnologyModelPeriodActivityLowerLimit[r,t] ;
```

In [ ]:
con = total_technology_model_period_activity >= ds['TotalTechnologyModelPeriodActivityLowerLimit']
mask = ds['TotalTechnologyModelPeriodActivityLowerLimit'] > 0
tac3_total_model_horizen_technology_activity_lower_limit = m.add_constraints(con, name='TAC3_TotalModelHorizenTechnologyActivityLowerLimit', mask=mask)

In [ ]:
tac3_total_model_horizen_technology_activity_lower_limit

## Emissions

```ampl
s.t. E1_AnnualEmissionProductionByMode{r in REGION, t in TECHNOLOGY, e in EMISSION, m in MODE_OF_OPERATION, y in YEAR:
									   EmissionActivityRatio[r,t,e,m,y] <> 0}:
	EmissionActivityRatio[r,t,e,m,y] * TotalAnnualTechnologyActivityByMode[r,t,m,y]
	=
	AnnualTechnologyEmissionByMode[r,t,e,m,y];
```


In [ ]:
if ~ds.coords['EMISSION'].isnull():
    mask = ~ds['EmissionActivityRatio'].isnull()
    con = ds['EmissionActivityRatio'] * total_annual_technology_activity_by_mode - annual_technology_emission_by_mode == 0
    e1_annual_emission_production_by_mode = m.add_constraints(con, name='E1_AnnualEmissionProductionByMode', mask=mask)

In [ ]:
# e1_annual_emission_production_by_mode

```ampl
s.t. E2_AnnualEmissionProduction{r in REGION, t in TECHNOLOGY, e in EMISSION, y in YEAR}:
	sum{m in MODE_OF_OPERATION}
	AnnualTechnologyEmissionByMode[r,t,e,m,y]
	=
	AnnualTechnologyEmission[r,t,e,y];
```

In [ ]:
if ~ds.coords['EMISSION'].isnull():
    con = annual_technology_emission_by_mode.sum(dims='MODE_OF_OPERATION') - annual_technology_emission == 0
    e2_annual_emission_production = m.add_constraints(con, name='E2_AnnualEmissionProduction')

In [ ]:
# e2_annual_emission_production

```ampl
s.t. E6_EmissionsAccounting1{r in REGION, e in EMISSION, y in YEAR}:
	sum{t in TECHNOLOGY}
	AnnualTechnologyEmission[r,t,e,y]
	=
	AnnualEmissions[r,e,y];
```

In [ ]:
if ~ds.coords['EMISSION'].isnull():
    con = annual_technology_emission.sum(dims=['TECHNOLOGY']) - annual_emissions == 0
    e6_emissions_accounting = m.add_constraints(con, name='E6_EmissionsAccounting1')

In [ ]:
# e6_emissions_accounting

# Objective Function
```ampl
minimize cost: sum{r in REGION, y in YEAR} TotalDiscountedCost[r,y];
```

In [ ]:
objective = total_discounted_cost.sum(dims=['REGION', 'YEAR'])
m.add_objective(expr=objective)

# Solving

In [ ]:
m.to_file('simplicity.lp')

In [ ]:
print(available_solvers)

In [ ]:
# m.solve()

In [ ]:
m.variables.get_name_by_label(61)

In [ ]:
m.constraints.get_name_by_label(43)

In [ ]:
m.solution[['RateOfProduction']].to_dataframe()

In [ ]:
m.solution['OperatingCost'].to_dataframe()

In [ ]:
m.solution['DiscountedSalvageValue'].to_dataframe()

In [ ]:
m.solution['TotalDiscountedCost'].to_dataframe()

In [ ]:
from get_vars import parse_gmpl_code

with open('OSeMOSYS.txt', 'r') as textfile:
    osemosys = textfile.readlines()

    sets, params, vars = parse_gmpl_code("".join(osemosys))

In [ ]:
with open('auto_osemosys.py', 'w') as write_file:
    for name, variable in vars.items():
        lower = -inf
        upper = inf
        integer = False
        indices = [x.split(' in ')[-1].strip() for x in variable['indices']]
        if variable.get('bounds'):
            bounds = variable['bounds'].split('=')
            sign = bounds[0].strip()
            value = bounds[1].strip()
            if sign == '>':
                lower = value
            elif sign == '<':
                upper = value
        if variable.get('integer'):
            integer = True

        coords = ", ".join(["ds.coords['" + str(x) + "']" for x in indices])
        variable_name = ''.join('_'+c.lower() if c.isupper() else c for c in name).strip('_')
        write_file.write((f"coords = [{coords}]\n"))
        write_file.write(f"{variable_name} = m.add_variables(lower={lower}, upper={upper}, coords=coords, name='{name}', integer={integer})\n")
